In [1]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libxtst6
Suggested packages:
  libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 10 not upgraded.
Need to get 28.3 MB of archives.
After this operation, 104 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libxtst6 amd64 2:1.2.3-1 [12.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 openjdk-8-jre-headless amd64 8u372-ga~us1-0ubuntu1~20.04 [28.3 MB]
Fetched 28.3 MB in 2s (17.2 MB/s)                 
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 109859 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1

In [2]:
#from google.colab import drive
import pandas as pd
import pyspark
from pyspark.sql import SparkSession

#link to google drive
#drive.mount('/content/gdrive')
spark = SparkSession.builder.appName('term project 1').config("spark.executor.memory", "40G").config("spark.driver.memory","40G")\
                  .config("spark.executor.cores","10")\
                  .config("spark.python.worker.memory","40G")\
                  .config('spark.plugins','com.nvidia.spark.SQLPlugin')\
                  .getOrCreate()
spark.conf.set('spark.rapids.sql.enabled','true')
spark.conf.set('spark.rapids.sql.incompatibleOps.enabled', 'true')
spark.conf.set('spark.rapids.sql.format.csv.read.enabled', 'true')
spark.conf.set('spark.rapids.sql.format.csv.enabled', 'true')
spark.conf.set('spark.storage.memotyFraction','true')



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/27 09:46:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.functions import monotonically_increasing_id

data = spark.read.csv('/kaggle/input/flightdata/itineraries.csv', sep=',', inferSchema=True, header=True)
data = data.withColumn("index", monotonically_increasing_id())

In [4]:
# import oil and holiday data
data_oil = spark.read.csv('/kaggle/input/flightdata/Jet Fuel Spot Price FOB (Dollars per Gallon) (1).csv', sep=',', inferSchema=True, header=True)
data_holiday = spark.read.csv('/kaggle/input/flightdata/Holidays (1).csv', sep=',', inferSchema=True, header=True)

In [ ]:
##########process itineraries########
# drop null and duplicate
data = data.na.drop()
data = data.na.drop(subset=["totalTravelDistance"])
data = data.drop("searchDate").dropDuplicates()

In [7]:
from pyspark.sql.functions import size, col, udf, struct
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import split
from datetime import datetime

def CountDistinctCol(*args):
    count = 0
    distinct_list = []
    for arg in args:
        if arg is not None and arg not in distinct_list :
            distinct_list.append(arg)
    return len(distinct_list)

def getArrivalTime(*args):
    arrival_time = None
    for arg in reversed(args):
        if arg is not None:
            dt = datetime.fromisoformat(str(arg))
            arrival_time = str(dt.hour) + ":" + str(dt.minute)
            break
    return arrival_time

def getDeptTime(*args):
    dept_time = None
    for arg in args:
        if arg is not None:
            dt = datetime.fromisoformat(str(arg))
            dept_time = str(dt.hour) + ":" + str(dt.minute)
            break
    return dept_time

def SumCol(*args):
    total = 0
    for arg in args:
        if arg is not None:
            total += int(arg)
    return total

CountDistinctCol_Udf = udf(CountDistinctCol, IntegerType())
SumCol_Udf = udf(SumCol, IntegerType())
getDeptTime_Udf = udf(getDeptTime, StringType())
getArrivalTime_Udf = udf(getArrivalTime, StringType())

In [8]:


data_airport_count = data.select("index", size(split(col("segmentsArrivalAirportCode"), "\\|\\|")).alias("segmentsArrivalAirportCode_Count"),\
                                              size(split(col("segmentsDepartureAirportCode"), "\\|\\|")).alias("segmentsDepartureAirportCode_Count"))

data_additional = data.select("index",\
              split("segmentsAirlineCode", "\|\|").getItem(0).alias("segmentsAirlineCode_1"),\
              split("segmentsAirlineCode", "\|\|").getItem(1).alias("segmentsAirlineCode_2"),\
              split("segmentsAirlineCode", "\|\|").getItem(2).alias("segmentsAirlineCode_3"),\
              split("segmentsAirlineCode", "\|\|").getItem(3).alias("segmentsAirlineCode_4"),\
              split("segmentsAirlineCode", "\|\|").getItem(4).alias("segmentsAirlineCode_5"),\
              split("segmentsEquipmentDescription", "\|\|").getItem(0).alias("segmentsEquipmentDescription_1"),\
              split("segmentsEquipmentDescription", "\|\|").getItem(1).alias("segmentsEquipmentDescription_2"),\
              split("segmentsEquipmentDescription", "\|\|").getItem(2).alias("segmentsEquipmentDescription_3"),\
              split("segmentsEquipmentDescription", "\|\|").getItem(3).alias("segmentsEquipmentDescription_4"),\
              split("segmentsEquipmentDescription", "\|\|").getItem(4).alias("segmentsEquipmentDescription_5"),\
              split("segmentsDurationInSeconds", "\|\|").getItem(0).alias("segmentsDurationInSeconds_1"),\
              split("segmentsDurationInSeconds", "\|\|").getItem(1).alias("segmentsDurationInSeconds_2"),\
              split("segmentsDurationInSeconds", "\|\|").getItem(2).alias("segmentsDurationInSeconds_3"),\
              split("segmentsDurationInSeconds", "\|\|").getItem(3).alias("segmentsDurationInSeconds_4"),\
              split("segmentsDurationInSeconds", "\|\|").getItem(4).alias("segmentsDurationInSeconds_5"),\
              split("segmentsCabinCode", "\|\|").getItem(0).alias("segmentsCabinCode_1"),\
              split("segmentsCabinCode", "\|\|").getItem(1).alias("segmentsCabinCode_2"),\
              split("segmentsCabinCode", "\|\|").getItem(2).alias("segmentsCabinCode_3"),\
              split("segmentsCabinCode", "\|\|").getItem(3).alias("segmentsCabinCode_4"),\
              split("segmentsCabinCode", "\|\|").getItem(4).alias("segmentsCabinCode_5"),\
            split("segmentsDepartureTimeRaw", "\|\|").getItem(0).alias("segmentsDepartureTimeRaw_1"),\
              split("segmentsDepartureTimeRaw", "\|\|").getItem(1).alias("segmentsDepartureTimeRaw_2"),\
              split("segmentsDepartureTimeRaw", "\|\|").getItem(2).alias("segmentsDepartureTimeRaw_3"),\
              split("segmentsDepartureTimeRaw", "\|\|").getItem(3).alias("segmentsDepartureTimeRaw_4"),\
              split("segmentsDepartureTimeRaw", "\|\|").getItem(4).alias("segmentsDepartureTimeRaw_5"),\
              split("segmentsArrivalTimeRaw", "\|\|").getItem(0).alias("segmentsArrivalTimeRaw_1"),\
              split("segmentsArrivalTimeRaw", "\|\|").getItem(1).alias("segmentsArrivalTimeRaw_2"),\
              split("segmentsArrivalTimeRaw", "\|\|").getItem(2).alias("segmentsArrivalTimeRaw_3"),\
              split("segmentsArrivalTimeRaw", "\|\|").getItem(3).alias("segmentsArrivalTimeRaw_4"),\
              split("segmentsArrivalTimeRaw", "\|\|").getItem(4).alias("segmentsArrivalTimeRaw_5"),\
              split("flightDate", "-").getItem(0).alias("flightYear").cast(IntegerType()),\
              split("flightDate", "-").getItem(1).alias("flightMonth").cast(IntegerType()),\
              split("flightDate", "-").getItem(2).alias("flightDay").cast(IntegerType()))


columns_to_count = ["segmentsAirlineCode_1", "segmentsAirlineCode_2","segmentsAirlineCode_3","segmentsAirlineCode_4","segmentsAirlineCode_5",]
data_additional = data_additional.withColumn("segmentsAirlineCode_Count", CountDistinctCol_Udf(*data_additional.columns[1:6]))
columns_to_count = ["segmentsEquipmentDescription_1", "segmentsEquipmentDescription_2","segmentsEquipmentDescription_3","segmentsEquipmentDescription_4","segmentsEquipmentDescription_5",]
data_additional = data_additional.withColumn("segmentsEquipmentDescription_Count", CountDistinctCol_Udf(*data_additional.columns[6:11]))
columns_to_count = ["segmentsDurationInSeconds_1", "segmentsDurationInSeconds_2","segmentsDurationInSeconds_3","segmentsDurationInSeconds_4","segmentsDurationInSeconds_5",]
data_additional = data_additional.withColumn("segmentsDurationInSecondse_Count", SumCol_Udf(*data_additional.columns[11:16]))
columns_to_count = ["segmentsCabinCode_1", "segmentsCabinCode_2","segmentsCabinCode_3","segmentsCabinCode_4","segmentsCabinCode_5",]
data_additional = data_additional.withColumn("segmentsCabinCode_Count", CountDistinctCol_Udf(*data_additional.columns[16:21]))
columns_to_count = ["segmentsDepartureTimeRaw_1", "segmentsDepartureTimeRaw_2","segmentsDepartureTimeRaw_3","segmentsDepartureTimeRaw_4","segmentsDepartureTimeRaw_5",]
data_additional = data_additional.withColumn("DepartureTime", getDeptTime_Udf(*data_additional.columns[21:26]))
columns_to_count = ["segmentsArrivalTimeRaw_1", "segmentsArrivalTimeRaw_2","segmentsArrivalTimeRaw_3","segmentsArrivalTimeRaw_4","segmentsArrivalTimeRaw_5",]
data_additional = data_additional.withColumn("ArrivalTime", getArrivalTime_Udf(*data_additional.columns[26:31]))

#data_additional.show(10)
#data_additional_test.show(10)

In [9]:
data_additional = data_additional.withColumn("DeptHrs", split("DepartureTime", ":").getItem(0).cast(IntegerType()))\
                    .withColumn("DeptMin", split("DepartureTime", ":").getItem(1).cast(IntegerType()))\
                    .withColumn("ArrHrs", split("ArrivalTime", ":").getItem(0).cast(IntegerType()))\
                    .withColumn("ArrMin", split("ArrivalTime", ":").getItem(1).cast(IntegerType()))

In [10]:
data_additional = data_additional.select("index", "segmentsAirlineCode_Count", "segmentsEquipmentDescription_Count", "segmentsDurationInSecondse_Count","segmentsCabinCode_Count",\
                                        "DeptHrs", "DeptMin", "ArrHrs", "ArrMin","flightMonth","flightDay")
data_additional = data_additional.join(data_airport_count, ["index"])

In [11]:
# combine data
from pyspark.sql.functions import lit
from pyspark.sql.functions import col

data_joined = data.join(data_additional, ["index"])


In [ ]:
##### for holiday and oil ######
def ConvertToDict(list1, list2):
  dict_list = {}
  for i in range(0, len(list1)):
    dict_list[list1[i]] = list2[i]
  return dict_list

In [12]:
#########process oil################
# convert date in oil to yyyy-mm-dd format
from pyspark.sql.functions import to_date, date_format

date = to_date("Date", "MMM dd, yyyy")
data_oil = data_oil.withColumn("Date_Formatted", date_format(to_date("Date", "MMM dd, yyyy"), "yyyy-MM-dd"))
#data_oil.show()



In [13]:
data_oil = data_oil.filter((data_oil['Date_Formatted'] >= "2022-04-01") & (data_oil['Date_Formatted'] < "2023-01-01"))

In [14]:
# combine with oil data
from pyspark.sql.functions import when

oil_date = data_oil.select('Date_Formatted').rdd.flatMap(lambda x : x ).collect()
oil_price = data_oil.select('`U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB (Dollars per Gallon)`').rdd.flatMap(lambda x:x).collect()
oil_list = ConvertToDict(oil_date, oil_price)

In [15]:
# fill in missing oil price for date
from datetime import datetime

idx = pd.date_range('2022-04-01', '2023-01-01')
tmp_series = pd.Series(oil_list)
tmp_series.index = pd.DatetimeIndex(tmp_series.index)
tmp_series = tmp_series.reindex(idx, fill_value=0)


prev = 0.0

for index, value in tmp_series.items():
    if value == 0:
      tmp_series._set_value(index, prev)
    else:
      prev = value

tmp_series.index = tmp_series.index.strftime('%Y-%m-%d')

# convert series to dict
oil_dict = tmp_series.to_dict()


In [16]:
from pyspark.sql import functions as F
from itertools import chain

lookup_map = F.create_map(*[F.lit(x) for x in chain(*oil_dict.items())])
lookup_map
data_joined = data_joined.withColumn("OilPrice", lookup_map[F.col("flightDate")])

#data_joined.show()

In [17]:
###########process holiday###########
# define airport location
from pyspark.sql import functions as F
from itertools import chain

loc_series = pd.Series({"City of Chicago":"ORD", "Los Angeles Unified" : "LAX","Miami-Dade County" : "MIA",\
                        "New York City" : "JFK","New York City" : "LGA", "Federal" : "all"})

# missing airport
# ATL, BOS, CLT, DEN, DFW, DTW, EWR, IAD, OAK, PHL, SFO

# convert series to dict
airport_dict = loc_series.to_dict()
lookup_map = F.create_map(*[F.lit(x) for x in chain(*airport_dict.items())])
data_holiday = data_holiday.withColumn("Airport", lookup_map[F.col("HolidayType")])


In [18]:
rdd = data_holiday.rdd.collect()

In [19]:
# convert holiday dataframe to dict

idx = pd.date_range('2022-01-01', '2023-01-01')
tmp_series = pd.Series({"2022-01-01" : False})
tmp_series.index = pd.DatetimeIndex(tmp_series.index)
tmp_series = tmp_series.reindex(idx, fill_value=False)
tmp_series.index = tmp_series.index.strftime('%Y-%m-%d')

start_date = []
end_date = []
for row in rdd:
    value = row.StartDate.strftime('%Y-%m-%d')
    start_date.append(value)
    value2 = row.EndDate.strftime('%Y-%m-%d') 
    end_date.append(value2)

for i in range(len(start_date)):
  for key in tmp_series.index:
      if start_date[i] <= key <= end_date[i]:
        #print(key)
        tmp_series[key] = True

#print(tmp_series)

holiday_dict = tmp_series.to_dict()
#holiday_dict


In [20]:
lookup_map = F.create_map(*[F.lit(x) for x in chain(*holiday_dict.items())])
data_joined = data_joined.withColumn("Holiday", lookup_map[F.col("flightDate")])

In [22]:
#data_joined.write.option("header",True).csv("/content/gdrive/MyDrive/data_output.csv")

In [23]:
##### start feature selection ######

In [24]:
#data_joined2 = data_joined.filter((data_joined.flightDate >= "2022-04-15") & (data_joined.flightDate <= "2022-06-15") & (data_joined.index <= 10000))
data_joined2, test_cand = data_joined.randomSplit([0.01, 0.9], seed=500)

In [25]:
# select features
data_selected = data_joined2.select("startingAirport", "destinationAirport", "fareBasisCode", "travelDuration","elapsedDays", "isBasicEconomy", "isRefundable", "isNonStop", "totalFare",\
                             "seatsRemaining", "segmentsArrivalAirportCode_Count", "segmentsAirlineCode_Count", "segmentsAirlineCode_Count", \
                             "segmentsEquipmentDescription_Count", "segmentsCabinCode_Count", "OilPrice","Holiday", "DeptHrs", "DeptMin", "ArrHrs", "ArrMin","flightMonth","flightDay")

data_selected.dtypes

[('startingAirport', 'string'),
 ('destinationAirport', 'string'),
 ('fareBasisCode', 'string'),
 ('travelDuration', 'string'),
 ('elapsedDays', 'int'),
 ('isBasicEconomy', 'boolean'),
 ('isRefundable', 'boolean'),
 ('isNonStop', 'boolean'),
 ('totalFare', 'double'),
 ('seatsRemaining', 'int'),
 ('segmentsArrivalAirportCode_Count', 'int'),
 ('segmentsAirlineCode_Count', 'int'),
 ('segmentsAirlineCode_Count', 'int'),
 ('segmentsEquipmentDescription_Count', 'int'),
 ('segmentsCabinCode_Count', 'int'),
 ('OilPrice', 'double'),
 ('Holiday', 'boolean'),
 ('DeptHrs', 'int'),
 ('DeptMin', 'int'),
 ('ArrHrs', 'int'),
 ('ArrMin', 'int'),
 ('flightMonth', 'int'),
 ('flightDay', 'int')]

In [27]:
from pyspark.sql.types import StringType

data_selected = data_selected.withColumn("isBasicEconomy",col("isBasicEconomy").cast(StringType()))\
           .withColumn("isRefundable", col("isRefundable").cast(StringType()))\
           .withColumn("isNonStop", col("isNonStop").cast(StringType()))\
           .withColumn("Holiday", col("Holiday").cast(StringType()))

#data_selected = data_selected.withColumn("isBasicEconomy", col("isBasicEconomy").cast(StringType()))\
#          .withColumn("Holiday", col("Holiday").cast(StringType()))

In [28]:
# split dataset to categorical and numerical
categorical = [x for (x, dataType) in data_selected.dtypes if dataType == "string"]
numerical = [x for (x, dataType) in data_selected.dtypes if ((dataType == "int") | (dataType == "double")) & (x != "totalFare")]
#print(categorical)
#print(numerical)

In [29]:
# do one hot encoder on categorical 
from pyspark.ml.feature import OneHotEncoder, StringIndexer

# string indexer transfor string label to index label
# create string indexer for each column in categorical
string_indexer = [StringIndexer(inputCol=x, outputCol=x+"_StringIndexer", handleInvalid="skip") for x in categorical]

# encode string index column to one hot encoder column
ohe = [OneHotEncoder(inputCols=[f"{x}_StringIndexer" for x in categorical],\
                    outputCols=[f"{x}_OneHotEncoder" for x in categorical],)]

In [30]:
from pyspark.ml.feature import VectorAssembler

assemblerInput = [x for x in numerical]
assemblerInput += [f"{x}_OneHotEncoder" for x in categorical]
#assemblerInput += ["flightDate"]

#assemblerInput

In [31]:
vector_assembler = VectorAssembler(inputCols=assemblerInput, outputCol="VectorAssembler_feature", handleInvalid='skip')


In [32]:
# define stage for pipeline
stages = []
stages += string_indexer 
stages += ohe 
stages += [vector_assembler]

In [33]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.01, labelCol="Delay", featuresCol='VectorAssembler_feature')



In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import MinMaxScaler

scaler = MinMaxScaler(inputCol="VectorAssembler_feature", outputCol="feature_scaled", min=0, max=100)
pipeline = Pipeline(stages = stages + [scaler])
pipeline_model = pipeline.fit(data_selected)
#data_transformed = pipeline_model.transform(d_train)

23/05/27 09:51:43 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
data_transformed = pipeline_model.transform(data_selected)
#data_transformed.show()

In [ ]:
#get only vector feature and label
data_transformed_final = data_transformed.select("feature_scaled","totalFare")
#data_transformed_final.show()

In [ ]:
###### feature selected by ml ######
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from  pyspark.ml.evaluation import RegressionEvaluator

classifier = RandomForestRegressor(labelCol="totalFare", featuresCol='feature_scaled')
paramGrid = (ParamGridBuilder()\
            .addGrid(classifier.maxDepth, [2, 5, 10])
            .addGrid(classifier.maxBins, [5, 10 , 20])
            .addGrid(classifier.numTrees,[5, 20, 50])
            .build())
crossval = CrossValidator(estimator=classifier,
                         estimatorParamMaps=paramGrid,
                         evaluator=RegressionEvaluator(labelCol='totalFare', predictionCol='prediction', metricName='mse'),
                         numFolds=2)
fitModel = crossval.fit(data_transformed_final)
BestModel= fitModel.bestModel
featureImportances= BestModel.featureImportances.toArray()

In [ ]:
# plot importance of feature
import matplotlib.pyplot as plt

x_values = list(range(len(featureImportances)))
plt.figure(figsize=(20,10));
plt.bar(x_values, featureImportances, orientation = 'vertical')
feature_list = ["startingAirport", "destinationAirport", "fareBasisCode", "travelDuration","elapsedDays", "isBasicEconomy", "isRefundable", "isNonStop",
                 "seatsRemaining", "segmentsArrivalAirportCode_Count", "segmentsAirlineCode_Count", "segmentsAirlineCode_Count",
                 "segmentsEquipmentDescription_Count", "segmentsCabinCode_Count", "OilPrice","Holiday", "DeptHrs", "DeptMin", "ArrHrs", "ArrMin","flightMonth","flightDay"]
plt.xticks(x_values, feature_list, rotation=90)
plt.ylabel('Importance')
plt.xlabel('Feature')
plt.title('Feature Importances');

In [ ]:
#for n in range(10, maximum, 10):
 #   print("Testing top n= ", n, " features")
    
#best_n_features= featureImportances.argsort()[-10:][::-1]
#best_n_features= best_n_features.tolist()



#### need to save vs
#vs= VectorSlicer(inputCol='features',   outputCol='best_features',indices=best_n_features)
#bestFeaturesDf= vs.transform(final_data)

'''    vs= VectorSlicer(inputCol='features',   outputCol='best_features',indices=best_n_features)
    bestFeaturesDf= vs.transform(final_data)
    train,test= bestFeaturesDf.randomSplit([0.7, 0.3])
    
    columns = ['Classifier', 'Result']
    vals = [('Place Holder', 'N/A')]
    results = spark.createDataFrame(vals, columns)
    paramGrid = (ParamGridBuilder()\
            .addGrid(classifier.maxDepth, [2, 5, 10])
            .addGrid(classifier.maxBins, [5, 10 , 20])
            .addGrid(classifier.numTrees,[5, 20, 50])
            .build())
crossval = CrossValidator(estimator=classifier,
                         estimatorParamMaps=paramGrid,
                         evaluator=MulticlassClassificationEvaluator(),
                         numFolds=2)
fitModel = crossval.fit(train)
BestModel= fitModel.bestModel
    featureImportances= BestModel.featureImportances.toArray()
    print("Feature Importances: ", featureImportances)
predictions = fitModel.transform(test)
accuracy = (MC_evaluator.evaluate(predictions))*100
print(" ")
print("Accuracy: ", accuracy)
'''
